# 1. Read data from mongoDB

In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

Check ip address https://whatismyipaddress.com/vi-vn/index?fbclid=IwAR1i50O9Py5PZk7UlnVXVHz2BU4MuPpQcnewuPg6KBJMx8oC9gPcFs5SbGg

In [2]:
password = "minhquan"
uri = "mongodb+srv://minhquan:{}@cluster0.fkdmnkx.mongodb.net/?retryWrites=true&w=majority".format(password)

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

db = client['Task2_Database']
collection = db['Shopee_Data']

data = list(collection.find())


Pinged your deployment. You successfully connected to MongoDB!


In [3]:
# Read data from MongoDB and save it in a list
cursor = collection.find()
data_list = []
for document in cursor: 
    del document['_id']
    data_list.append(document)


df = pd.DataFrame(data_list)
df.head()

KeyboardInterrupt: 

# 2. Extract data

## 2.1 Shape Data

In [ ]:
print(f'Dữ liệu có {df.shape[1]} cột')
print(f'Dữ liệu có {df.shape[0]} dòng')

Dữ liệu có 145 cột
Dữ liệu có 98295 dòng


## 2.2 Drop column unnecessary

In [ ]:
# column None value
non_columns=list(df.isna().sum()[df.isna().sum() == 8361].index)
df.drop(non_columns,axis=1,inplace=True)
columns = list(df.isna().sum()[df.isna().sum() > 0].index)
columns.remove('discount')
df.drop(columns,axis=1,inplace=True)
# column Nan value
count_Nan=(df =='').sum()
column_nan_value=list(count_Nan[count_Nan>0].index)
df.drop(column_nan_value,axis=1,inplace=True)

In [ ]:
# column unnecessary
unneccess_column=['label_ids','image','images','currency','status','ctime','liked','flag','cb_option',
                  'show_discount','discount','video_info_list','item_type','is_adult','badge_icon_type','shopee_verified','is_official_shop',
                  'show_official_shop_label','show_shopee_verified_label','show_official_shop_label_in_title','is_cc_installment_payment_eligible',
                  'is_non_cc_installment_payment_eligible','bundle_deal_id','can_use_bundle_deal','welcome_package_type','can_use_wholesale',
                  'is_preferred_plus_seller','has_model_with_available_shopee_stock','is_mart','is_service_by_shopee','repurchase_rate',
                  'best_selling_tag','info','data_type','key','count','adsid','campaignid','video_display_control','can_use_cod','pub_context_id',
                  'show_flash_sale_label','catid','has_lowest_price_guarantee','raw_discount']

df.drop(unneccess_column,axis=1,inplace=True)

## 2.3 Column
Mô tả các cột dữ liệu và các kiểu dữ liệu mong muốn

|STT| Tên cột    | Mô tả| Kiểu dữ liệu mong muốn |
|---|:-----------:|:-------------:|:---|
|1| itemid| Id của sản phẩm| String|
|2| shopid| ID của shop| string|
|3| stock| Số lượng sản phẩm trong kho| int |
|4| sold| Số lượng sản phẩm đã bán| int|
|5| liked_count| Số lượt thích sản phẩm| int|
|6| cmt_count| Số lượt bình luận| int|
|7| item_status| trạng thái sản phẩm| bool|
|8| price| giá sản phẩm| float|
|9| price_min|giá tối thiểu| float|
|10| price_max| giá tối đa| float|
|11| price_min_before_discount| giá tối thiểu trước discount| float|
|12| price_max_before_discount| giá tối đa trước discount| float|
|13| price_before_discount| giá trước discount| string|
|14| tier_variations| kích thước, màu sắc và các đặc tính khác của sản phẩm| dict|
|15| item_rating| các đánh giá của sản phẩm(bao gồm đánh giá trung bình và số đánh giá của từng sao)| dict|
|16| show_free_shipping| shop có free ship hay không| bool|
|17| shop_location|vị trí của shop| string|
|18| is_on_flash_sale| sản phẩm có đang sale hay không| bool|
|19| shop_name| tên shop| string|
|20| shop_rating| đánh giá của shop| float|
|21| global_sold_count| số lượng sản phẩm được bán trên thế giới trên shoppe| int|
|22| flash_sale_stock| số lượng sản phẩm còn lại trong đợt flash sale| string|
|23| crawl_time| thời điểm crawl data| date time|
|24| historical _sold| Số lượng đã bán trong quá khứ| int|
|25| name| tên sản phẩm| string|


# 3. Convert data type
Có nhiều cột có kiểu dữ liệu không hợp lý nên chuyển chúng về kiểu dữ liệu phù hợp với việc phân tích

## 3.1 price columns
các cột giá cả bị lớn hơn 100000 lần

In [ ]:
price_columns = ['price','price_min','price_max','price_min_before_discount','price_max_before_discount','price_before_discount']
df[price_columns]=df[price_columns]/100000

## 3.2 Interger type columns

In [ ]:
## Lấy các cột cùng dữ liệu
int_columns = []
float_columns = []
bool_columns = []
object_columns = []
for i in df.columns:
    if df[i].dtype == 'float64':
        float_columns.append(i)
    elif df[i].dtype == 'int64':
        int_columns.append(i)
    elif df[i].dtype == 'bool':
        bool_columns.append(i)
    else:
        object_columns.append(i)

In [ ]:
df[int_columns].describe()

,itemid,shopid,stock,sold,liked_count,cmt_count,global_sold_count,flash_sale_stock
count,9.829500e+04,9.829500e+04,9.829500e+04,98295.000000,98295.000000,98295.000000,98295.000000,98295.000000
mean,1.664161e+10,3.310714e+08,3.827543e+05,31.472079,535.796795,220.196907,763.053685,3.014019
std,7.255949e+09,3.648912e+08,9.632103e+06,247.485126,3030.083482,1383.506597,5259.320884,40.786592
min,5.343859e+07,1.620236e+06,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.225875e+10,6.861376e+07,2.900000e+01,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.957811e+10,1.270493e+08,2.460000e+02,0.000000,3.000000,2.000000,7.000000,0.000000
75%,2.238144e+10,7.440602e+08,6.000000e+03,4.000000,39.000000,26.000000,83.000000,0.000000
max,2.595037e+10,9.991228e+08,4.999982e+08,9437.000000,63258.000000,41401.000000,233410.000000,2459.000000


In [ ]:
# cột itemid và shopid nên ở dạng str vì nó không cần các tính toán số học
df['itemid']=df['itemid'].astype('str')
int_columns.remove('itemid')
df['shopid']=df['shopid'].astype('str')
int_columns.remove('shopid')
# Các cột còn lại chỉ cần int32 là đủ
df[int_columns]=df[int_columns].astype('int32')
df[int_columns].head()

,stock,sold,liked_count,cmt_count,global_sold_count,flash_sale_stock
0,159,161,41,40,163,159
1,240,3098,5112,12119,39098,240
2,222,439,77,136,462,222
3,216,387,107,118,397,216
4,317,26,35,87,227,0


## 3.3 Float columns

In [ ]:
df[float_columns].describe()

,price,price_min,price_max,price_min_before_discount,price_max_before_discount,price_before_discount,shop_rating
count,9.829500e+04,9.829500e+04,9.829500e+04,9.829500e+04,9.829500e+04,9.829500e+04,98295.000000
mean,2.911587e+05,2.911587e+05,3.045103e+05,2.617260e+05,2.808576e+05,2.617260e+05,4.695165
std,1.978006e+05,1.978006e+05,1.948049e+05,2.286351e+05,2.415443e+05,2.286351e+05,0.301730
min,2.500000e+03,2.500000e+03,2.500000e+03,-1.000000e-05,-1.000000e-05,0.000000e+00,4.016393
25%,1.550000e+05,1.550000e+05,1.700000e+05,-1.000000e-05,-1.000000e-05,0.000000e+00,4.640632
50%,2.390000e+05,2.390000e+05,2.590000e+05,2.500000e+05,2.700000e+05,2.500000e+05,4.845110
75%,3.990000e+05,3.990000e+05,3.990000e+05,3.990000e+05,4.328940e+05,3.990000e+05,4.890028
max,3.200000e+06,3.200000e+06,3.200000e+06,1.400000e+06,1.400000e+06,1.400000e+06,4.955559


In [ ]:
# giá cả và rating thì không thể âm được
df[float_columns]=df[float_columns].map(lambda x: 0 if x < 0 else x)
# có vẻ 3 cột price_min_before_discount, price_max_before_discount, price_before_discount có giá trị giống nhau
def check_2columns(column1, column2):
    return (column1 - column2).sum()
for i in range(len(float_columns)):
    for j in range(i,len(float_columns)):
        if float_columns[i] == float_columns[j]:
            continue
        else:
            rs=check_2columns(df[float_columns[i]],df[float_columns[j]])
            if rs == 0:
                print(float_columns[i],'trùng với',float_columns[j])
                
# drop price_min column và price_min_before_discount column
df.drop('price_min',axis=1,inplace=True)
float_columns.remove('price_min')
df.drop('price_min_before_discount',axis=1,inplace=True)
float_columns.remove('price_min_before_discount')
df[float_columns].head()

price trùng với price_min
price_min_before_discount trùng với price_before_discount


,price,price_max,price_max_before_discount,price_before_discount,shop_rating
0,285000.0,285000.0,598000.0,598000.0,4.855072
1,185000.0,185000.0,350000.0,350000.0,4.855071
2,185000.0,185000.0,450000.0,450000.0,4.855071
3,229000.0,229000.0,450000.0,450000.0,4.855071
4,99000.0,99000.0,438000.0,438000.0,4.855070


## 3.4 Object columns

In [ ]:
df[object_columns].head()

,item_status,tier_variations,item_rating,shop_location,shop_name,crawl_time
0,normal,"[{'name': 'MÀU SẮC', 'options': ['BTT122 ĐEN -...","{'rating_star': 4.925, 'rating_count': [40, 0,...",Hà Nội,Aviano Menswear,2023-11-02 12:00:00
1,normal,"[{'name': 'Màu', 'options': ['BTT09 Ghi Sáng',...","{'rating_star': 4.845550602608552, 'rating_cou...",Hà Nội,Aviano Menswear,2023-11-02 12:00:00
2,normal,"[{'name': 'MÀU SẮC', 'options': ['BTT118 ĐEN -...","{'rating_star': 4.897058823529412, 'rating_cou...",Hà Nội,Aviano Menswear,2023-11-02 12:00:00
3,normal,"[{'name': 'MÀU SẮC', 'options': ['AKG09 ĐEN', ...","{'rating_star': 4.872881355932203, 'rating_cou...",Hà Nội,Aviano Menswear,2023-11-02 12:00:00
4,normal,"[{'name': 'Màu Sắc', 'options': ['B045 Nâu', '...","{'rating_star': 4.875, 'rating_count': [88, 1,...",Hà Nội,Aviano Menswear,2023-11-02 12:00:00


In [ ]:
# hàm lấy ra màu
colors = ["ĐỎ","VÀNG","TRẮNG","ĐEN","HỒNG","TÍM","CAM","NÂU","XÁM","LAM",'GHI','XANH','BE','THAN','RÊU','CHÌ','NGẪU','TIÊU']

def is_color(list_color:str):
    list_return = []
    #item = item.split('.')[0]
    for item in list_color:
        item=item.replace(',',' ')
        item=item.replace('XDƯƠNG','XANH')
        item=item.replace('XANHĐÁ-Mũ','XANH')
        split_item = item.split(' ')
        for color in split_item:
            if '-' in color:
                color.replace(" ","")
                a = color.split('-')
                for cl in a:
                    if cl.upper() in colors:
                        list_return.append(cl)
            else:
                if color.upper() in colors:
                    list_return.append(color)
        if len(list_return) == 0:
            return 'NGẪU NHIÊN'
        return list_return
    
def get_options_size(columns):
    if len(columns) > 1:
        return columns[1]['options']
    return ['No size']

def get_options_color(columns):
        return columns[0]['options']

def explore_color(array):
    if type(array) == list:
        return ", ".join(array)
    else:
        return array
# color
df['color']=df['tier_variations'].apply(get_options_color)
df['color']=df['color'].apply(is_color)
df['color']=df['color'].apply(explore_color)
# size
df['size']=df['tier_variations'].apply(get_options_size)
# drop tier_variations columns
df.drop('tier_variations',axis=1,inplace=True)
# remove columns
object_columns.append('color')
object_columns.append('size')
object_columns.remove('tier_variations')

In [ ]:
# lấy rating
def get_rating_star(item):
    return item['rating_star']
def get_rating_count(item):
    return item['rating_count']
def get_rcount_with_image(item):
    return item['rcount_with_image']
def get_rcount_with_context(item):
    return item['rcount_with_context']

# init new column
df['rating_star'] = df['item_rating'].apply(get_rating_star)
df['rating_count'] = df['item_rating'].apply(get_rating_count)
df['rcount_with_image'] = df['item_rating'].apply(get_rcount_with_image)
df['rcount_with_context'] = df['item_rating'].apply(get_rcount_with_context)

object_columns.append('rating_star')
object_columns.append('rating_count')
object_columns.append('rcount_with_image')
object_columns.append('rcount_with_context')
object_columns.remove('item_rating')

In [ ]:
# write csv file

df.to_csv("new_data.csv")